In [3]:
import mne
import numpy as np
import matplotlib.pyplot as plt
from itertools import compress
import random

from scipy.signal import stft, istft
from scipy.signal.windows import hann, gaussian
from scipy.special import gamma
from scipy.optimize import curve_fit
import pywt

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

import math
from sklearn.linear_model import LinearRegression


In [4]:
# Control variables to be put here to be varied?
time_win = 5
sc_factor = 1

In [19]:
# Preparing the dataset that we're going to use
fnirs_folder = mne.datasets.fnirs_motor.data_path()
fnirs_cw_amplitude_dir = fnirs_folder / "Participant-1"
raw_intensity = mne.io.read_raw_nirx(fnirs_cw_amplitude_dir, verbose=True)
raw_intensity.load_data()



raw_intensity.annotations.set_durations(time_win)
raw_intensity.annotations.rename(
    {"1.0": "Control", "2.0": "Tapping/Left", "3.0": "Tapping/Right"}
)
unwanted = np.nonzero(raw_intensity.annotations.description == "15.0")
raw_intensity.annotations.delete(unwanted)
# Apparently channel 15 is unrelated to the motor activation experiment,
# it was used to signal something unrelated and so will be ignored



# Removes short channels
picks = mne.pick_types(raw_intensity.info, meg = False, fnirs = True)
dists = mne.preprocessing.nirs.source_detector_distances(
    raw_intensity.info, picks = picks)
raw_intensity.pick(picks[dists > 0.01])



# Converting to optical density based on readings, then to haemo readings
raw_od = mne.preprocessing.nirs.optical_density(raw_intensity)
raw_haemo = mne.preprocessing.nirs.beer_lambert_law(raw_od, ppf=6)

# We filter them to remove the frequencies associated with cardiac activity (unrelated)
raw_haemo_unfiltered = raw_haemo.copy()
raw_haemo.filter(0.05, 0.7, h_trans_bandwidth = 0.2, l_trans_bandwidth = 0.02)



# Creates epochs for each occurence of an event (Left tap, right tap, control)
reject_criteria = dict(hbo=80e-6)
tmin, tmax = -1, 25
events, event_dict = mne.events_from_annotations(raw_haemo)


epochs = mne.Epochs(
    raw_haemo,
    events,
    event_id=event_dict,
    tmin=tmin,
    tmax=tmax,
    reject=reject_criteria,
    reject_by_annotation=True,
    proj=True,
    baseline=(None, 0),
    preload=True,
    detrend=None,
    verbose=True,
)

isolated_event_epochs = epochs["Tapping"]
control_event_set = epochs["Control"]
isol_dhrf_data = isolated_event_epochs.get_data(copy=True)
avg_haemo_func_tapping = isolated_event_epochs.average()
std_dev = np.std(avg_haemo_func_tapping.data, axis=0)

Loading /Users/kostasdemiris/mne_data/MNE-fNIRS-motor-data/Participant-1
Reading 0 ... 23238  =      0.000 ...  2974.464 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.05 - 0.7 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.05
- Lower transition bandwidth: 0.02 Hz (-6 dB cutoff frequency: 0.04 Hz)
- Upper passband edge: 0.70 Hz
- Upper transition bandwidth: 0.20 Hz (-6 dB cutoff frequency: 0.80 Hz)
- Filter length: 1291 samples (165.248 s)

Used Annotations descriptions: ['Control', 'Tapping/Left', 'Tapping/Right']
Not setting metadata
90 matching events found
Setting baseline interval to [-1.024, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 90 events and 204 original time po

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished


In [ ]:
# Conversion between the different forms i need, including for display purposes.
class convs:
    def __init__(self):
        pass
    
    def apply_stft(self, signal, sample_rate):
        window = gaussian(128, std = 7)
        hop = 128
        # returns in the format: frequency, time, result
        return stft(signal, sample_rate, nperseg = 64)
    
    def wavelet_decmp(self, signal, m_wavelet, d_level):
         return pywt.wavedec(signal, m_wavelet, d_level)
    
    def i_wavelet_decmp(self, coeffs, m_wavelet):
        return pywt.waverec(coeffs, m_wavelet)
    
    def inv_WvImg_conv(self, wvImg, arr_shape, minim_data, maxim_data):
        # Firstly seperate it into channels, for each row of pixels
        # remove all padded values, un-normalise then inverse_wavelet_conv
        un_padded = []
        un_norm = rev_normalise(wvImg, minim_data, maxim_data)
        for row in un_norm:
            un_padded.append(self.i_wavelet_decmp(rem_padding(row, arr_shape), wavelet))
        return un_padded
    
    def padarray(self, arr, size): 
        return np.pad(arr, pad_width=(0, size-len(arr)), mode='constant')
    
    def rem_padding(self, arr, arr_shape):
        original_array = []
        sec = max(arr_shape)
        start_i = 0
        while start_i * sec < len(arr) - 1:
            original_array.append(arr[start_i * sec : (start_i * sec) + arr_shape[start_i]])
            start_i += 1
        return original_array
    
    def normalise(self, arr, scale_factor=sc_factor):
        data_min = np.min(arr)
        data_max = np.max(arr)
        return ((arr - data_min) / (data_max - data_min)) * scale_factor

    def rev_normalise(self, arr, n_min, n_max, scale_factor=sc_factor):
        return (((arr-1)/scale_factor) * (n_max - n_min)) + n_min
    
    # Not necessary if using the base normalise function, just for testing
    def event_to_wvImg(self, sample, wavelet, level, min_n, max_n):
        wv_channel_arr = []
        for channel in sample:
            wv_form = convs.wavelet_decmp(channel, wavelet, level)
            maxim_len = max(len(sub_arr) for sub_arr in wv_form)
            wv_channel_arr.append(np.array([padarray(arr, maxim_len) for arr in wv_form]).flatten())
        wv_channel_arr = np.array(wv_channel_arr)
        normalised = sc_factor * (wv_channel_arr - min_n) / ( max_n - min_n)
        normalised[wv_channel_arr == 0] = -1
        return normalised
    
    def convert_signal(self, signal):
        decomposed = convs.wavelet_decmp(signal, wavelet, level)
        pad_decomp = np.array([padarray(a[i], len(a[len(a)-1])) for i in range(len(a))]).flatten()
        return pad_decomp

    def reverse_conversion(self, signal, sig_min, sig_max, sub_arr_lens):
        unNorm = rev_normalise(signal, sig_min, sig_max)
        unpadded_Ver = rem_padding(unNorm, sub_arr_lens)
        return convs.i_wavelet_decmp(unpadded_Ver, wavelet)
    
    def z_curve_normalisation(self, data):
        return (data - np.mean(data)) / np.std(data)
    

convs = convs()

In [21]:
class display:
    def __init__(self, time_sig):
        self.time_signal = time_sig
    
    def display_gray_arr(self, img_arr):
        plt.imshow(img_arr, cmap='gray')
        plt.title("Wavelet as Image")
        plt.axis('off')
        plt.show()
    
    def plot_pChannel(self, points, time_series):
        plt.figure(figsize=(10, 4))
        plt.plot(time_series, points)
        plt.title("channel")
        plt.xlabel("Time [s]")
        plt.ylabel("Amplitude")
        plt.show()
    
    def plot_scatter_data(self, data_points, central=0.5):
        plt.scatter([i for i in range(len(data_points))], data_points, color='white', alpha=0.5)

        for i in range(len(data_points)):
            plt.plot([i, i], [data_points[i], central], color='red', alpha=0.5)

        plt.axhline(y=central, color='green', linestyle='--')

        plt.title('Scatter Plot of Data Points by Index')
        plt.xlabel('Index')
        plt.ylabel('Value')
        plt.grid(True)
        plt.show()
    

time_signal = np.arange(len(avg_haemo_func_tapping.data[0])) * (1/signal_rate)      
displays = display(time_signal)

In [22]:
# The actual conversion of the data to wavelet form and so forth, for the 60 original signals

wavelet = 'sym4' # Resembles cannonical haemo-dynamic responce 
level = 4
first_transform = convs.wavelet_decmp(isol_dhrf_data[0][0], wavelet, level)
lengths = [len(sub_array) for sub_array in first_transform]
max_length = max(lengths)
signal_rate = 7.81
indv_coeff_len = len(first_transform)


n_data = np.empty((60, 40, (level+1) * max_length))

for i in range(0, len(isol_dhrf_data)):
    for j in range(0, len(isol_dhrf_data[0])):
        temp = convs.wavelet_decmp(isol_dhrf_data[i][j], wavelet, level)
        n_data[i][j] = np.array([convs.padarray(arr, max_length) for arr in temp]).flatten()
        
n_min = np.min(n_data)
n_max = np.max(n_data)
norm_data = convs.normalise(n_data)  

In [23]:
# Learnt this off a tutorial that I need to put credit on here at some point
class AutoRegression:
    def __init__(self, order):
        self.order = order 
        # The order of an AutoRegression model is the number of previous variables used to calculate the next one
        self.model = LinearRegression()
        self.std = None
        
    def generate_predictor_data(self, data_set):
        n = len(data_set)
        
        data = data_set[:n - self.order]
        data = np.reshape(data, (-1, 1))
        
        # This takes the values from index 0 up to n-order and reshapes them into a column vector 
        # We then stack the values from index 1 to (n-order) + 1. 
        
        # If we hadn't done 0 independently, we'd have nothing to stack onto
        for i in range(1, self.order):
            temp_col = data_set[i:(n+i) - self.order]
            temp_col = np.reshape(temp_col, (-1, 1))
            np.hstack((data, temp_col))
            
        return data
    
    def generate_responce_data(self, data_set):
        return data_set[self.order:]
    
    def fit(self, data_set):
        self.std = np.std(data_set)
        t_x_data = self.generate_predictor_data(data_set)  # X data for training
        t_y_data = self.generate_responce_data(data_set)  # Y data for training
        self.model.fit(t_x_data, t_y_data)
        
    def predict(self, data_set, step_count, simulation_num):
        data_set = np.array(data_set)
        output = np.array([])
        
        for sim in range(simulation_num):
            # We do a monte carlo simulation approach, performing a bunch of simulations, then averaging them
            temp_ans = []
            tape = data_set[-self.order:]
            # Take the last order num values as a rolling tape measure, then keep rolling back to the start
            # along the tape, putting in our predicted values in as we go
            
            for prediction in range(step_count):
                predicted = self.model.predict(np.reshape(tape, (-1, 1)))
                predicted += np.random.normal(loc=0, scale=self.std)
                # We predict the next value, then add some gaussian noise to it depending on the standard
                # deviation of the original dataset we were using...
                
                temp_ans.append(predicted[0])
                tape = np.roll(tape, -1) 
                # We move the tape backwards by one, the last value becomes the second to last and so forth
                tape[-1] = predicted[0]
                
            if sim == 0:
                output = np.array(temp_ans)
            else:
                output = output + np.array(temp_ans)
                # since we'll be averaging out the values anyway, just add them elementwise for now
        output = output/ simulation_num
        return output    
    
    
auto_regression = AutoRegression(30)

In [62]:
class random_noise_gen:
    def __init__(self):
        pass
    
    def generate_freq(self, frequency, sample_rate, sig_length, amplitude, start_offset=0):
        points = []
        interval = 1 / sample_rate
        curr = 0
        while curr + interval <= sig_length:
            points.append(amplitude * math.sin(frequency * 2 * math.pi * (curr + start_offset)))
            curr += interval
        return points
    
    def generate_noise(self, frequencies, sample_rate, sig_length, max_amp):
        noise = np.zeros(math.floor(sample_rate * sig_length))
        for freq in frequencies:
            noise = noise + self.generate_freq(freq, sample_rate, sig_length, random.uniform(0, max_amp), start_offset=random.uniform(0, math.pi))
        return noise
    
    def generate_white_noise(self, p_num, amplitude=1):
        output = np.zeros((p_num))
        for i in range(p_num):
            output[i] = np.random.random_sample()
        return output
    
    def generate_rolling_noise(self, mean_freq, std_freq, mean_amp, std_amp, sig_length, sample_rate, starting_offset=1):
        # Generates noise that changes over time in a "rolling" manner - we impose gradually changing trends on it
        output = np.zeros((sig_length+20,))
        sum_weighted_time = starting_offset
        for i in range(len(output)):
            output[i] = (math.sin(2 * math.pi * sum_weighted_time) + 1) * np.random.normal(mean_amp, std_amp)
            sum_weighted_time += np.random.normal(mean_freq, std_freq) * (1 / sample_rate)
        for i in range(10, len(output)-10):
            output[i] = sum(output[i-10:i+10]) / 20
        return output[10: len(output)-10]
    
    # generates noise, then temporally correlates it with a base 33 order (overridable)
    def generate_tCorr_noise(self, p_num, order=33):
        noise_data = self.generate_white_noise(p_num * 2)
        auto_regression.fit(noise_data)
        time_correlated_data = auto_regression.predict(noise_data, p_num, order)
        return time_correlated_data
    

        
    
random_gen = random_noise_gen()

# displays.plot_scatter_data(noise_generator.generate_rolling_noise(0.1, 1, 0.5, 0.01, 1000, 10))
# displays.plot_scatter_data(noise_generator.generate_tCorr_noise(1000))


In [84]:
class bio_modelling:
    # This is for the modelling of the biological signals that we're dealing with, as opposed to the mostly mathematicaly stuff from the
    def __init__(self, generator):
        self.generator = generator
    
    @staticmethod
    def gamma_dist(t, k, theta):
        # For a specific time, it returns the value of a gamma distribution with a given k and theta at that time
        return  (t ** (k-1)) * (math.e ** (-t / theta)) / (gamma(k) * (theta** k))

    @staticmethod
    def bC_hrf(t, k1, k2, theta1, theta2):
        # returns the basic version of the canonical hrf, modelled as the difference between two gamma functions
        return bio_modelling.gamma_dist(t, k1, theta1) - bio_modelling.gamma_dist(t, k2, theta2)
    
    def generate_phony_signal(self, p_num, generator=random_gen):
        if generator is None:
            generator = self.generator
        time_correlated_points = (generator.generate_tCorr_noise(p_num) - 0.5) * math.pow(10, -9)
        heart_rate = generator.generate_rolling_noise(1, 0.25, 0, math.pow(10, -7), p_num, 7.81)
        respi_rate = generator.generate_rolling_noise(0.3, 0.1, 0, math.pow(10, -7), p_num, 7.81)
        mayer_wavs = generator.generate_rolling_noise(0.1, 0.005, 0, math.pow(10, -7), p_num, 7.81)
        return time_correlated_points + heart_rate + respi_rate + mayer_wavs
    
    def get_basic_gamma_params(self, target_signal, time_axis, initial_paramaters=None):
        parameters, covariance = curve_fit(bio_modelling.bC_hrf, time_axis, target_signal, p0=initial_parameters)
        # Gradient descent to fit parameters of the function to have it fit the curve
        k1, k2, theta1, theta2 = parameters
        return k1, k2, theta1, theta2
    
    def generate_noise_set(self, set_size, signal_length, noise_function=None):
        if noise_function is None:
            return np.array([self.generate_phony_signal(signal_length+30)[15: signal_length + 15] for point in range(set_size)])
        
        return np.array([noise_function(signal_length) for point in range(set_size)])
    
    def generate_gamma_set(self, time_axis, parameters, size, mean=0, var=0.01, canonical=True, magnitude_correction=1):
        # generates a set of gamma functions, applying some necessary limitation to the varying if the hrf is canonical
        gen_params = np.zeros((size, 4))
        
        for set in range(len(gen_params)):
            
            # There's a better way to do this, for each set just generate a new parameter set instead of storing them all
            # Fix this later
            if canonical:
                while gen_params[i][0] >= gen_params[i][1]:
                    gen_params[i][0] = parameters[0] + np.random.normal(mean, var) # k1
                    gen_params[i][1] = parameters[1] + np.random.normal(mean, var) # k2
                while gen_params[i][2] >= gen_params[i][3]:
                    gen_params[i][2] = parameters[2] + np.random.normal(mean, var) # theta1
                    gen_params[i][3] = parameters[3] + np.random.normal(mean, var) # theta2
            else:
                gen_params[i][0] = parameters[0] + np.random.normal(mean, var) # k1
                gen_params[i][1] = parameters[1] + np.random.normal(mean, var) # k2
                gen_params[i][2] = parameters[2] + np.random.normal(mean, var) # theta1
                gen_params[i][3] = parameters[3] + np.random.normal(mean, var) # theta2
        
        gamma_set = [[bio_modelling.bC_hrf(time_axis, gen_params[i][0], gen_params[i][1], gen_params[i][2], gen_params[i][3])] for i in range(size)]
        return magnitude_correction * np.array(gamma_set)
        
    
models = bio_modelling(random_gen)

In [88]:
# Actually generating the data
signal_number = 10000
signal_length = len(time_signal)
initial_parameters = [3, 5, 2, 1]
p_mean, p_var = 0, 0.01


# Currently we'll be working on the first channel only, for simplicity's sake
gamma_params = models.get_basic_gamma_params(avg_haemo_func_tapping.data[0], time_signal, initial_parameters)
print("this is the length of gamma params", len(gamma_params))
clean_data = models.generate_gamma_set(time_signal, gamma_params, signal_number, magnitude_correction=1.2*1e-5)
print("this is the length of clean data", len(clean_data))
noise_data = models.generate_noise_set(signal_number//5, signal_length)
print("this is the length of noise data", len(noise_data))
null_data = np.zeros((signal_number - (signal_number//5),) + np.shape(noise_data)[1:])
print("This is the shape of the null_data", np.shape(null_data))


this is the length of gamma params 4
this is the length of clean data 10000


/var/folders/vt/8vz112hj0ms1db0lb3xcvmch0000gn/T/ipykernel_96673/746220001.py:9: RuntimeWarning: divide by zero encountered in reciprocal
  return  (t ** (k-1)) * (math.e ** (-t / theta)) / (gamma(k) * (theta** k))
/var/folders/vt/8vz112hj0ms1db0lb3xcvmch0000gn/T/ipykernel_96673/746220001.py:9: RuntimeWarning: divide by zero encountered in divide
  return  (t ** (k-1)) * (math.e ** (-t / theta)) / (gamma(k) * (theta** k))
/var/folders/vt/8vz112hj0ms1db0lb3xcvmch0000gn/T/ipykernel_96673/746220001.py:9: RuntimeWarning: invalid value encountered in divide
  return  (t ** (k-1)) * (math.e ** (-t / theta)) / (gamma(k) * (theta** k))


this is the length of noise data 2000
This is the shape of the null_data (8000, 204)


In [ ]:
baseline_data = np.concatenate((noise_data, null_data))
np.random.shuffle(baseline_data)
print("This is the length of the base_line data", len(baseline_data))
overlayed_data = clean_data + baseline_data
print("we done...")


This is the length of the base_line data 10000


In [ ]:
# Preprocesssing the data
z_overlay = convs.z_curve_normalisation(overlayed_data)
z_clean = convs.z_curve_normalisation(overlayed_data)


In [ ]:
# Dataset classes for encoer
basic_transform = transforms.Compose([
    transforms.ToTensor()
])


# Dataset for training with data as a set of amplitude points
class amp_dataset:
    def __init__(self, clean, overlayed, transform):
        assert(len(clean) == len(overlayed))
        self.clean = clean
        self.overlayed = overlayed
        self.transform = transform
    
    def __len__(self):
        return len(self.overlayed)
    
    def __getitem__(self, idx):
        if self.transform is None:
            self.transform = lambda x: x
            
        noisy = self.transform(overlayed[idx])
        clean = self.transform(clean[idx])
        
        return noisy, clean
    
amp_set = amp_dataset(clean, overlayed_data, basic_transform)




In [ ]:
class Amp_Lin_AutoEncoder(nn.Module):
    # This is the linear neural network implementation of an auto-encoder using amplitude points
    def __init__(self, signal_length):
        # Signal length has to be longer than 200
        super().__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(signal_length, 124),
            nn.ReLU(),
            nn.Linear(124, 72),
            nn.ReLU(),
            nn.Linear(72, 54)
        )